In [17]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
from datetime import datetime
import os
import glob
from Model.imageProcess import f_image_read, f_image_preprocessing_010, f_save_image, f_imagesingle_read, f_image_box, f_image_box_license, f_image_box_license_2
from Model.sqlProcess import f_sql_object_detection

yolo_model = "yolov10m.pt"
YoloModelVehicle = YOLO(f"Yolo/{yolo_model}")
# Dapatkan mapping dari ID kelas ke nama kelas
class_names = YoloModelVehicle.names

# print(f"vehicle_results : {YoloModelVehicle.names}")

image_folder = 'ImgVehicle'
image_files = glob.glob(os.path.join(image_folder, '*.jpg'))

for image in image_files:
    tanggal_start = datetime.today()
    print(f"image : {image}")

    frame = cv2.imread(image)

    vehicle_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    original_frame = frame.copy()

    # Deteksi kendaraan
    vehicle_results = YoloModelVehicle(frame)

    # print(f"vehicle_results : {vehicle_results}")
    for vehicle in vehicle_results:
        # print(f"vehicle : {vehicle.boxes}")
        for box in vehicle.boxes:
            print(f"box : {box}")

            x1, y1, x2, y2 = map(int, box.xyxy[0].numpy())

            # print(f"box.cls[0] : {box.cls[0]}")

            class_id = int(box.cls[0])
            confidence = box.conf[0]

            if class_id in [2, 3, 5, 7] and confidence > 0.00:
                detected_class_label = class_names[class_id]

                # f_image_box(frame, x1, x2, y1, y2, "")

        # f_image_read(image1=original_frame, title1=f"{image} Original", image2=frame, title2="Object Box")

                tanggal_end = datetime.today()

                f_sql_object_detection(p_label_name=image, 
                                       p_yolo_model=yolo_model, 
                                       p_object_type=detected_class_label, 
                                       p_confidence=confidence, 
                                       p_start_dt=tanggal_start.strftime("%Y-%m-%d %H:%M:%S"), 
                                       p_end_dt=tanggal_end.strftime("%Y-%m-%d %H:%M:%S"))

image : ImgVehicle\bike (1).jpg

0: 640x640 1 person, 2 cars, 2 motorcycles, 456.0ms
Speed: 6.4ms preprocess, 456.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
box : ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.8882])
data: tensor([[6.3432e+01, 2.0168e-01, 1.8177e+02, 2.0974e+02, 8.8821e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (240, 240)
shape: torch.Size([1, 6])
xywh: tensor([[122.5997, 104.9712, 118.3346, 209.5390]])
xywhn: tensor([[0.5108, 0.4374, 0.4931, 0.8731]])
xyxy: tensor([[6.3432e+01, 2.0168e-01, 1.8177e+02, 2.0974e+02]])
xyxyn: tensor([[2.6430e-01, 8.4033e-04, 7.5736e-01, 8.7392e-01]])
box : ultralytics.engine.results.Boxes object with attributes:

cls: tensor([3.])
conf: tensor([0.8314])
data: tensor([[ 50.1553,  72.2603, 190.9202, 226.4515,   0.8314,   3.0000]])
id: None
is_track: False
orig_shape: (240, 240)
shape: torch.Size([1, 6])
xywh: tensor([[120.5377, 149.3559, 140.7649, 154.1912]]